<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/RAG/Evaluating_RAG_pipeline_with_RAGAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using the same notebook as in Personal resource assistant. For explanation of code till Evaluation kindly check out that notebook first.

## Installing necessary libraries

In [1]:
!pip install langchain-cohere langchain pdfminer.six chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00

## Importing libraries

In [2]:
import os
from google.colab import userdata
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_KEY')
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_cohere import ChatCohere
from langchain.schema.output_parser import StrOutputParser
from pdfminer.high_level import extract_text as extract_text_pdf_miner
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## VectorDB setup

In [3]:
persist_directory = "/content/chroma_db"

embedding = CohereEmbeddings(
    model="embed-english-v3.0",
    user_agent="langchain"
)

for pdf_name in ["/content/Newwhitepaper_Agents.pdf", "/content/Newwhitepaper_Operationalizing Generative AI on Vertex AI.pdf"]:
    with open(pdf_name, 'rb') as f:
        text = extract_text_pdf_miner(f)
        cleaned_text = " ".join(text.split("\n"))
        docs = []
        splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)
        for chunk in splitter.split_text(cleaned_text):
            docs.append(Document(page_content=chunk, metadata={"source": pdf_name}))
    vector_collection_fixed_size = Chroma.from_documents(
        documents=docs,
        persist_directory=persist_directory,
        embedding=embedding
    )

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

<ipython-input-3-880a7a0b4320>:3: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embedding = CohereEmbeddings(
<ipython-input-3-880a7a0b4320>:22: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


## RAG pipeline

In [5]:
llm = ChatCohere(model="command-r")

prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """

prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context": vectordb.as_retriever(),
        "question": RunnablePassthrough()
    }
)

output_parser = StrOutputParser()

chain = retrieval | prompt | llm | output_parser

In [6]:
response = chain.invoke("What is AI agents?")
print(response)

AI agents, or Generative AI agents, are applications that use a combination of reasoning, logic, and access to external information to achieve specific goals. They are designed to work autonomously and can act independently, leveraging various tools and cognitive architectures to observe the world, make decisions, and take action. These agents can access real-time information, suggest real-world actions, and execute complex tasks without human intervention. AI agents are particularly useful for messy pattern recognition tasks.


## Evaluation

In [7]:
!pip install ragas==0.2.0 langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
sample_queries = [
    "What is the score of YOLO on VOC 2012 test set?",
    "How many layers does the Encoder of Transformer is composed of?",
]

# This are the one liner responses that humans might respond to these questions
# While an LLM might produce a bigger response, our need if to identify that the response by LLM does not exclude the actual answer
expected_responses = [
    "On the VOC 2012 test set, YOLO scores 57.9% mAP.",
    "The encoder is composed of a stack of N = 6 identical layers",
]

In [9]:
dataset = []

for query,reference in zip(sample_queries,expected_responses):

    relevant_docs = vectordb.similarity_search(query)
    response = chain.invoke(query)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":[doc.page_content for doc in relevant_docs],
            "response":response,
            "reference":reference
        }
    )

In [10]:
dataset

[{'user_input': 'What is the score of YOLO on VOC 2012 test set?',
  'retrieved_contexts': ['Imagine the output   to a given input is factually inaccurate. How can you find out which of the components are   the ones that didn’t perform well? To answer this question it is necessary to apply logging   on the application level and at the component level. We need lineage in our logging for all   components executed. For every component we need to log their inputs and outputs. We   also need to be able to map those with any additional artifacts and parameters they depend   on so we can easily analyze those inputs and outputs.  Monitoring can be applied to the overall gen AI application and to individual components. We   prioritize monitoring at the application level. This is because if the application is performant   and monitoring proves that, it implies that all components are also performant. You can also   apply the same practices to each of the prompted model components to get more gra

In [11]:
from datasets import Dataset
evaluation_dataset = Dataset.from_list(dataset)

In [12]:
evaluation_dataset

Dataset({
    features: ['user_input', 'retrieved_contexts', 'response', 'reference'],
    num_rows: 2
})

### Use the same LLM or different LLM like OpenAI to evaluate

In [13]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata
OpenAI_API = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OpenAI_API
open_ai_llm = ChatOpenAI(model="gpt-4o-mini")

The 3 metrics which we will be using


1. LLMContextRecall: Evaluates how well retrieved contexts align with claims in the reference answer, estimating recall without manual reference context annotations.

2. Faithfulness: Assesses whether all claims in the generated answer can be inferred directly from the provided context.

3. Factual Correctness: Checks the factual accuracy of the generated response by comparing it with a reference, using claim-based evaluation and natural language inference.

In [14]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

evaluator_llm = LangchainLLMWrapper(open_ai_llm)

result = evaluate(
    llm=evaluator_llm,
    dataset = evaluation_dataset,
    metrics=[
        LLMContextRecall(llm=evaluator_llm),
        FactualCorrectness(llm=evaluator_llm),
        Faithfulness(llm=evaluator_llm)
    ]
)

df = result.to_pandas()

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
display(df.head())

,user_input,retrieved_contexts,response,reference,context_recall,factual_correctness,faithfulness
0,What is the score of YOLO on VOC 2012 test set?,[shown for a variety of detection methods. YOL...,YOLO scores 57.9% mAP on the VOC 2012 test set.,"On the VOC 2012 test set, YOLO scores 57.9% mAP.",1.0,1.0,1.0
1,How many layers does the Encoder of Transforme...,[mechanism instead of sequence- aligned recurr...,The Encoder of the Transformer is composed of ...,The encoder is composed of a stack of N = 6 id...,1.0,1.0,1.0
